## Imports

In [43]:
import requests
import pandas as pd
from bs4 import BeautifulSoup
from tqdm import tqdm
import googlemaps
import numpy as np
import re
import os
import folium

## PD Set Options

In [41]:
pd.set_option('display.max_columns', 500)
pd.set_option('display.max_colwidth', None)
pd.set_option('display.max_rows', 800)

## Data read-in

In [31]:
df = pd.read_excel('Copy of 230209.2021CCAORetentionPINs.xlsx')

## Data clean

In [32]:
df['PARID'] = df['PARID'].str.replace('-','')

## Test block

In [33]:
sample_pin = '17162050160000'
response = requests.get(f'https://www.cookcountyassessor.com/pin/{sample_pin}')
soup = BeautifulSoup(response.content, 'html.parser')
address = soup.find('span', text='Address').find_next('span').text
print(address)

10 S DEARBORN ST


## Set up scraper

In [34]:
def address_snagger(PARID):
    PIN = PARID['PARID']
    try:
        response = requests.get(f'https://www.cookcountyassessor.com/pin/{PIN}')
        soup = BeautifulSoup(response.content, 'html.parser')
        address = soup.find('span', text='Address').find_next('span').text
        return address
    except Exception as e:
        return PIN

In [35]:
# df['address'] = df.apply(address_snagger, axis=1)

In [36]:
address_list = [address_snagger(row) for row in tqdm(df.to_dict('records'))]


 17%|███████▏                                  | 96/559 [01:22<06:47,  1.14it/s]


 35%|██████████████▏                          | 193/559 [02:59<07:22,  1.21s/it]


 52%|█████████████████████▎                   | 290/559 [04:35<04:45,  1.06s/it]


 69%|████████████████████████████▍            | 387/559 [06:10<02:54,  1.01s/it]


 87%|███████████████████████████████████▍     | 484/559 [07:46<01:11,  1.04it/s]


100%|█████████████████████████████████████████| 559/559 [09:02<00:00,  1.03it/s]


In [37]:
df['ADDRESS'] = address_list

In [40]:
df.to_csv('Assessors_hit_list_with_addresses.csv')

# Make Map

## Geocoder Setup

In [44]:
%store -r google_maps_API_Key
gmaps_key = googlemaps.Client(key=google_maps_API_Key)

In [46]:
### add Cook County, IL to geo_address ###
df['geo_address'] = df['ADDRESS'] + ' Cook County, IL'

In [48]:
### Define and run geocoder ###
def geocode(add):
    g = gmaps_key.geocode(add)
    lat = g[0]["geometry"]["location"]["lat"]
    lng = g[0]["geometry"]["location"]["lng"]
    return (lat, lng)

df['geocoded'] = df['geo_address'].apply(geocode)

In [50]:
df['geocoded'] = df['geocoded'].astype(str)
df[['lat', 'lon']] = df['geocoded'].str.strip('()').str.split(', ', expand=True)
df['lat'] = df['lat'].astype(float)
df['lon'] = df['lon'].astype(float)

## HTML Popup Formatter

In [51]:
df.columns

Index(['PARID', '2021 BOR Final \nAssessed Value',
       '2022 CCAO Initial \nAssessed Value', '$ Difference', '% Difference2',
       'Class', 'Township', 'ADDRESS', 'geo_address', 'geocoded', 'lat',
       'lon'],
      dtype='object')

In [52]:
def popup_html(row):
    Address = row['ADDRESS']
    BOR_Final_2021 = row['2021 BOR Final \nAssessed Value']
    CCAO_Initial_Assessed_2022 = row['2022 CCAO Initial \nAssessed Value']
    difference = row['$ Difference']
    percent_difference = row['% Difference2']
    class_ = row['Class']
    township = row['Township']
    PIN = row['PARID']
    
    html = '''<!DOCTYPE html>
    <html>
    <strong>Address: </strong>{}'''.format(Address) + '''<br>
    <strong>BOR Final 2021: </strong>{}'''.format(BOR_Final_2021) + '''<br>
    <strong>CCAO Initial Assessed 2022: </strong>{}'''.format(CCAO_Initial_Assessed_2022) + '''<br>
    <strong>Difference: </strong>{}'''.format(difference) + '''<br>
    <strong>% Difference: </strong>{}'''.format(percent_difference) + '''<br>
    <strong>% Class: </strong>{}'''.format(class_) + '''<br>
    <strong>% Township: </strong>{}'''.format(township) + '''<br>
    <strong>% PIN: </strong>{}'''.format(PIN) + '''<br>
    </html>
    '''
    return html

In [53]:
### Create map container ###
m = folium.Map(location=df[["lat", "lon"]].mean().to_list(),zoom_start=10)


### Create title ###
title_html = '''
              <h3 align="center" style="font-size:16px"><b>{}</b></h3>
             '''.format(f"INSERT TITLE HERE")

m.get_root().html.add_child(folium.Element(title_html))

for index, row in df.iterrows():
    marker = folium.Marker(
        location=[row['lat'], row['lon']],
        popup=folium.Popup(popup_html(row), max_width=400))
    marker.add_to(m)

# Display map
m

In [54]:
m.save('index.html')

## Map URL snagger